In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

In [ ]:
import os
import sys
import re
import datetime
import inspect
import bs4 as bs
import urllib.request
import pandas as pd
import matplotlib as mpl

In [ ]:
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
vi = sys.version_info
print("Python: {0}.{1}.{2}".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    mname = None
    if name.startswith("__"):
        continue
    if hasattr(mod, "__version__"):
        if hasattr(mod, "__path__"):
            mname = os.path.split(mod.__path__[0])[1]
        if mname and mname != name:
            print("{0} as {1}: {2}".format(mname, name, mod.__version__))
        else:
            print("{0}: {1}".format(name, mod.__version__))            
    elif hasattr(mod, "__file__") and "site-packages" in mod.__file__:
        if mname and mname != name:
            print("{0} as {1}: No __version__ ".format(mname, name))
        else:
            print("{0}: No __version__ ".format(name))            
del mod
del name

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_elections_in_the_United_States"
source = urllib.request.urlopen(url).read()
soup = bs.BeautifulSoup(source,'lxml')
#<table class="wikitable sortable jquery-tablesorter" style="text-align:right;">
tables = soup.findAll('table', attrs={'class':'wikitable'})
l = []
dfs ={}
table = tables[0]
table_rows = table.find_all('tr')
for tr in table_rows:
    th = tr.find('th').text.strip()
    td = tr.find_all('td')
    row = [th] + [tr.text.strip() for tr in td]
    l.append(row)
df = pd.DataFrame(l, )
df.drop(labels=6, axis=1, inplace=True)
columns = ["Year", "General", "Senate", "House", "President", "Governer"]
df.columns = columns
print(df.shape)
df.head()

In [ ]:
multi_hdr = [('Election', 'year'), ('Winner & party', 'Winner'), ('Winner & party', 'party'), 
             ('Electoral College', 'Votes'), ('Electoral College', 'Pct'), 
             ('Popular vote', 'Pct'), ('Popular vote', 'MarginPct'), 
             ('Popular vote', 'Votes'), ('Popular vote', 'Margin'), 
             ('Runner-up & party', 'Runner-up'), ('Runner-up & party', 'party'), 
             ('Turnout', 'Pct')]
df.columns = pd.MultiIndex.from_tuples(multi_hdr)
int_cols = [('Popular vote','Margin'), ('Popular vote','Votes')]
for ic in int_cols:
    df[ic] = df[ic].apply(lambda x: int(x.replace(',', '').replace("−", "-")))
df.loc[df[('Turnout', 'Pct')] == '-%',('Turnout', 'Pct')] = '65.4'
pct_cols = [('Electoral College', 'Pct'), ('Popular vote', 'Pct'),
            ('Popular vote', 'MarginPct'),('Turnout', 'Pct')]
for pc in pct_cols:
    try:
        df[pc] = df[pc].apply(lambda x: float(x.replace('%', '').replace("−", "-")))    
    except Exception as e:
        print(pc)
        raise Exception(e)
df.loc[df[('Election','year')] == '1788–89',('Election','year') ] = 1789
df.loc[df[('Election','year')] == '2020[Note 9]',('Election','year') ] = 2020
df[('Election','year')] = df[('Election','year')].apply(lambda x: int(x))        
print(df.head(2).to_string())
print()
print(df.tail(2).to_string())